# Solving 3D ising model on GPU

#### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba as nb
import scipy as sp
import torch as to
from matplotlib.animation import FuncAnimation
device = to.device('cuda' if to.cuda.is_available() else 'cpu')
from mpl_toolkits.mplot3d import Axes3D
import os

#### Initialize Grids

In [2]:
N = 150
random = to.rand((N,N,N)).to(device)
lat = to.zeros((N,N,N)).to(device)
lat[random >= 0.25 ] = 1
lat[random < 0.25 ] = -1
lat2 = lat.unsqueeze(0).unsqueeze(0)

random = to.rand((N,N,N)).to(device)
lat3 = to.zeros((N,N,N)).to(device)
lat3[random >= 0.75 ] = 1
lat3[random < 0.75 ] = -1
lat4 = lat.unsqueeze(0).unsqueeze(0)

#### Create a function for calculating the energies, energy changes at every position in the lattice

In [3]:
def energy_lat_gpu(lat):
    kernel = sp.ndimage.generate_binary_structure(3,1)
    kernel[1][1][1] = False

    kernel = to.tensor(kernel.astype(np.float32)).unsqueeze(0).unsqueeze(0).to(device)

    en_lat = -lat * to.nn.functional.conv3d(lat, kernel, padding="same")

    energy = en_lat.sum(dim=(1,2,3,4))

    dE_array = -2 * en_lat
    return energy, en_lat, dE_array

E, E_arr, dE_array = energy_lat_gpu(lat2)
E

tensor([-5027872.], device='cuda:0')

#### Creating our metropolis algorithm function

In [4]:
def metro_alg_gpu(lat_3D, steps, BJ):
    device = lat_3D.device
    energies = []
    avg_spin = []
    lat = lat_3D.clone()
    BJ = BJ.reshape([-1,1,1,1,1])
    lattice_snapshots = []
    for t in range(steps):
        i = to.randint(0, 2, (), device=device).item()
        j = to.randint(0, 2, (), device=device).item()
        k = to.randint(0, 2, (), device=device).item()

        _, _, dE_arr = energy_lat_gpu(lat)
        dE = dE_arr[:,:, i::2, j::2, k::2]

        rand_vals = to.rand_like(dE, device=device)

        change = (dE >= 0) * (rand_vals < to.exp(-BJ*dE)) + (dE < 0)
        lat[:,:, i::2, j::2, k::2][change] *= -1

        E, _, _ = energy_lat_gpu(lat)
        energies.append(E)
        avg_spin.append(lat.sum((1,2,3,4)) / N**3)
        if t % 2 == 0:
            lattice_snapshots.append(lat[0,0].cpu().clone())

    return to.vstack(avg_spin), to.vstack(energies), lattice_snapshots


#### Running the function and plotting the results for our algorithm run for one temperature

In [ ]:
BJ = 0.6*to.ones(lat2.shape[0]).to(device)
avgspin , energy , _ = metro_alg_gpu(lat2,10000,BJ)
fig, axes = plt.subplots(1, 2, figsize=(12,4))
ax = axes[0]
ax.plot(avgspin[:,0].cpu())
ax.set_xlabel('Algorithm Time Steps')
ax.set_ylabel(r'Average Spin $\bar{m}$')
ax.grid()
ax = axes[1]
ax.plot(energy[:,0].cpu())
ax.set_xlabel('Algorithm Time Steps')
ax.set_ylabel(r'Energy $E/J$')
ax.grid()
fig.tight_layout()
plt.show()

In [14]:
def MES_metro_gpu(lat,BJ,steps=500):
    lat = lat.unsqueeze(dim=0).repeat(len(BJ),1,1,1,1)
    spins, energies, lattice_snapshots = metro_alg_gpu(lat, steps, BJ)
    tail = int(0.4 * steps)
    avg_spin = spins[-tail:].mean(dim=0)
    energies_avg = energies[-tail:].mean(dim=0)
    energies_std = energies[-tail:].std(dim=0)
    
    return avg_spin, energies_avg, energies_std , lattice_snapshots

In [ ]:
BJs = 1/to.linspace(3, 5.5, 20).to(device)
spins_avg, E_means, E_std, lattice_snapshots = MES_metro_gpu(lat, BJs)

### Plotting Spins against temperature

In [ ]:
plt.plot(1/BJs.cpu(), spins_avg.cpu(), marker='o')
plt.show()

### Plotting Heat Capacity * Constants against temperature

In [ ]:
plt.plot(1/BJs.cpu(), E_std.cpu(), marker='o')
plt.show()